# The ***GroupBy*** Pattern

We have seen the ***GroupBy*** operator in ***Pandas***, but  this is actually a more general ***design pattern*** that can be utilized in many data analyics frameworks and data access interfaces, e.g. in ***SQL***. 

## GroupBy: general Pattern
<img SRC="IMG/groupby.jpg">

## GroupBy in SQL:
```
SELECT COUNT(CustomerID), Country
FROM Customers
GROUP BY Country
ORDER BY COUNT(CustomerID) DESC;
```

#### Pandas SQL-Query example
```
params=tuple([202101, 202102, 202103])

sql = f""" SELECT p.description, SUM(p.price) as price
          FROM product p
          WHERE p.extenal_id in {params}
          GROUP BY p.description """
result = pd.read_sql_query(sql, con=conn)

```


## GroupBy in Pandas

In [1]:
#setup example
import numpy as np
import pandas as pd
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.002275,0.257882
1,a,two,-1.171275,-0.207212
2,b,one,0.344921,0.758358
3,b,two,0.041823,0.344792
4,a,one,-0.004239,-0.674135


In [2]:
#group by key1 
grouped = df.groupby(df['key1'])
grouped #this is now a more complex group object

In [3]:
#and generates a table per group
for name, group in grouped:
    print ("name:", name, "\n",group)

name: a 
   key1 key2     data1     data2
0    a  one -0.002275  0.257882
1    a  two -1.171275 -0.207212
4    a  one -0.004239 -0.674135
name: b 
   key1 key2     data1     data2
2    b  one  0.344921  0.758358
3    b  two  0.041823  0.344792


In [4]:
#access group table
grouped.get_group('b')

,key1,key2,data1,data2
2,b,one,0.344921,0.758358
3,b,two,0.041823,0.344792


In [5]:
#get numper of entries (rows) per group
grouped.size()

key1
a    3
b    2
dtype: int64

In [6]:
#get number of group entries by columns
grouped.count()

,key2,data1,data2
key1,,,
a,3,3,3
b,2,2,2


#### Think of  grouped DataFrames as 3d objects:

In [7]:
#accessing the "3d" group tables
grouped['data2'].get_group('a')

0    0.257882
1   -0.207212
4   -0.674135
Name: data2, dtype: float64

In [8]:
grouped.get_group('a')['data2']

0    0.257882
1   -0.207212
4   -0.674135
Name: data2, dtype: float64

### Group by external keys

In [9]:
#define external key years as numpy array
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([years]).mean()


2005   -0.377242
2006    0.170341
Name: data1, dtype: float64

### Group by functions

In [10]:
#sort by column and retun top n
def top(df, n=5, column='data1'):
    return df.sort_values(by=column)[-n:]

df.groupby(df['key1']).apply(top,n=5)

key1 key2     data1     data2
key1                                
a    1    a  two -1.171275 -0.207212
     4    a  one -0.004239 -0.674135
     0    a  one -0.002275  0.257882
b    3    b  two  0.041823  0.344792
     2    b  one  0.344921  0.758358

### Group-wise aggregation (apply)
<img SRC="IMG/groupby.jpg">

#### Typical build in aggregation functions:
* sum
* mean
* max / min
* quantile
* ...

In [11]:
#aggregate over the groups
grouped.count()

,key2,data1,data2
key1,,,
a,3,3,3
b,2,2,2


In [12]:
grouped.sum()

,data1,data2
key1,,
a,-1.177789,-0.623464
b,0.386744,1.103150


#### Custom Aggregation Functions

In [13]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.169000,0.932017
b,0.303098,0.413566


#### Multiple aggregations

In [14]:
#just call a list of function
grouped.agg([peak_to_peak, 'mean', 'median'])

data1                            data2                    
     peak_to_peak      mean    median peak_to_peak      mean    median
key1                                                                  
a        1.169000 -0.392596 -0.004239     0.932017 -0.207821 -0.207212
b        0.303098  0.193372  0.193372     0.413566  0.551575  0.551575

#### Suppressing the Group Keys 

In [15]:
df.groupby(df['key1']).apply(top,n=2)

key1 key2     data1     data2
key1                                
a    4    a  one -0.004239 -0.674135
     0    a  one -0.002275  0.257882
b    3    b  two  0.041823  0.344792
     2    b  one  0.344921  0.758358

In [16]:
df.groupby(df['key1'], group_keys=False).apply(top,n=2)

,key1,key2,data1,data2
4,a,one,-0.004239,-0.674135
0,a,one,-0.002275,0.257882
3,b,two,0.041823,0.344792
2,b,one,0.344921,0.758358
